In [1]:
import tensorflow as tf
import numpy as np

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [56]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [57]:
reset_graph()

In [71]:
from datetime import datetime

def log_dir(prefix="", model=False, timestamp=False, subdirname=""):
    """
    - model - set True if model is saved (e.g. saver.save())
    - timestamp - set True to create unique directory
    """
    time = ""
    root_dir = "../models" if model else "../tf_logs"
    if prefix:
        prefix += "-"
    if timestamp:
        time = "-" + datetime.utcnow().strftime("%Y%m%d%H%M%S")
    name = prefix + "model" + time
    logdir = "{}/{}".format(root_dir, name) if not subdirname else "{}/{}/{}".format(root_dir, subdirname, name)
    return logdir

In [59]:
n_inputs = 28 * 28  # MNIST image
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [60]:
X = tf.placeholder(dtype=tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(dtype=tf.int32, shape=(None), name="y")

In [61]:
# simple function for a neuron layer - actually we don't need it since TF has its own implementation
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs + n_neurons)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [62]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [63]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [64]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [65]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [66]:
file_writer = tf.summary.FileWriter(log_dir(prefix="mnist_NN"), tf.get_default_graph())
file_writer.close()

Our graph is ready. Let's start a learning phase:

Prepare the data:

In [41]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
print(X_train.shape, X_train.dtype)
print(y_train.shape, y_train.dtype)
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0   # reshape and scale from 0.0 to 1.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_test = y_test.astype(np.int32)
y_train = y_train.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

(60000, 28, 28) uint8
(60000,) uint8


Start learning:

In [42]:
n_epochs = 40
batch_size = 50

In [44]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, batch_size):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [74]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print("Epoch {:2}\tBatch accuracy: {:.5f}\tVal accuracy: {:.5f}".format(epoch, acc_train, acc_val))
    save_path = saver.save(sess, log_dir(prefix="mnist_NN", model=True, subdirname="mnist_NN_model"))

Epoch  0	Batch accuracy: 0.58273	Val accuracy: 0.56600
Epoch  1	Batch accuracy: 0.71636	Val accuracy: 0.73340
Epoch  2	Batch accuracy: 0.79818	Val accuracy: 0.79740
Epoch  3	Batch accuracy: 0.82091	Val accuracy: 0.82940
Epoch  4	Batch accuracy: 0.82727	Val accuracy: 0.84940
Epoch  5	Batch accuracy: 0.83909	Val accuracy: 0.86220
Epoch  6	Batch accuracy: 0.86182	Val accuracy: 0.87000
Epoch  7	Batch accuracy: 0.84727	Val accuracy: 0.87740
Epoch  8	Batch accuracy: 0.87909	Val accuracy: 0.88400
Epoch  9	Batch accuracy: 0.87818	Val accuracy: 0.88680
Epoch 10	Batch accuracy: 0.88727	Val accuracy: 0.89040
Epoch 11	Batch accuracy: 0.88727	Val accuracy: 0.89480
Epoch 12	Batch accuracy: 0.89273	Val accuracy: 0.89780
Epoch 13	Batch accuracy: 0.90000	Val accuracy: 0.90120
Epoch 14	Batch accuracy: 0.90182	Val accuracy: 0.90440
Epoch 15	Batch accuracy: 0.90182	Val accuracy: 0.90500
Epoch 16	Batch accuracy: 0.91000	Val accuracy: 0.90900
Epoch 17	Batch accuracy: 0.89909	Val accuracy: 0.90980
Epoch 18	B

Use model:

In [75]:
with tf.Session() as sess:
    saver.restore(sess, save_path)
    X_new = X_test[:20]
    Z = logits.eval(feed_dict={X: X_new})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ../models/mnist_NN_model/mnist_NN-model


In [86]:
import pandas as pd

results = pd.DataFrame({"Actual": y_test[:20], "Prediction": y_pred})
results["Correct"] = results["Actual"] == results["Prediction"]
results

,Actual,Prediction,Correct
0,7,7,True
1,2,2,True
2,1,1,True
3,0,0,True
4,4,4,True
5,1,1,True
6,4,4,True
7,9,9,True
8,5,6,False
9,9,9,True
